IMPORT SOME LIBRARIES AND READ IN DATA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
##check out tutorial here:
##http://nbviewer.ipython.org/github/jvns/pandas-cookbook/blob/v0.1/cookbook/Chapter%201%20-%20Reading%20from%20a%20CSV.ipynb
df = pd.read_csv('train.csv')
df1 = df.drop('Winner', 1)
df['Win_bin'] = (df['Winner'] == 'Barack Obama') ##new column: logical wrt winner
df.head()

VISUALIZE THE DATA (1ST AND 2ND PRINCIPAL COMPONENTS)

In [ ]:
##using (parts of) andreas mueller's intro. to scikit-learn as a pseudo-template for analyzing 
##the POTUS data:
##https://github.com/amueller/sklearn_tutorial/blob/master/presentation.ipynb
from sklearn.decomposition import PCA #import principal component analysis
pca = PCA(n_components=2) #Instantiate the model & set parameters
pca.fit(df1); #Fit the model
df_pca = pca.transform(df1)#Apply the model. For embeddings / decompositions, this is transform.
print(df_pca.shape)
plt.scatter(df_pca[:, 0], df_pca[:, 1] , c = df['Win_bin'] , s = 50 , alpha=0.4);

SPLIT DATA INTO TRAINING AND TEST SETS

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1, df['Win_bin'], random_state=0)
print("X_train shape: %s" % repr(X_train.shape))
print("y_train shape: %s" % repr(y_train.shape))
print("X_test shape: %s" % repr(X_test.shape))
print("y_test shape: %s" % repr(y_test.shape))

LET'S TRY A LINEAR SVM

In [ ]:
from sklearn.svm import LinearSVC
svm = LinearSVC() # Instantiate the model.
svm.fit(X_train, y_train);#Fit the model.
svm.predict(X_train)# apply the model to the hold-out set
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

NOW LET'S TRY A SVM WITH A GAUSSIAN KERNEL

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf')# Instantiate the model (SVM w/ Gaussian kernel)
svm.fit(X_train, y_train);#Fit the model.
svm.predict(X_train)# apply the model to the hold-out set
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

NOW WE'RE GOING TO TRY A BUNCH OF SVMs (LINEAR, POLYNOMIAL & GAUSSIAN KERNELS) AND PERFORM
A GRID SEARCH OVER THE HYPERPARAMETERS (TUNING THEM), SELECTING FOR PRECISION

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import svm, metrics, preprocessing
import csv
import time
# Set the parameters by cross-validation

start_time = time.time()

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10]},
                    {'kernel': ['linear'], 'C': [1, 10]},{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['poly'], 'degree': [2]},
                    {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['poly'], 'degree': [3]},
                    {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['poly'], 'degree': [4]}]


scores = ['precision'] # you can alter this by adding, for example, `recall'

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=1), tuned_parameters)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_estimator_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %s"
              % (mean_score, scores.std() / 2, params))
    print()
    
print(time.time()- start_time)